#  Import stuff

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
#plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
import datetime
from IPython.display import clear_output
import time
import seaborn as sns
sns.set_style('darkgrid')
color = sns.color_palette()

# Read the csv data 

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## Convert date column to datetime type

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [ ]:
display(train.head())
display(test.head())

# Sort test & train data by store then date

In [ ]:
test = test.sort_values(by=['Store','Date'])
train = train.sort_values(by=['Store','Date'])

In [ ]:
train['Date'].unique().max(), train['Date'].unique().min() 



In [ ]:
test['Date'].unique().max(), test['Date'].unique().min() 

In [ ]:
test[test['Store'] == 1].head()

Get store list from needed for test data

In [ ]:
store_list = test['Store'].unique()

In [ ]:
train.head()

# Analyzing data

Stasionary test and acf, pacf graph

In [ ]:
store = 21

date_from = '2013-06-02'
date_to = ''
rsmpl_opt = '3d'
spqd_period = 24
program_starts = time.time()
iters = 0

sales = train[train['Store'] == store]
sales = sales.set_index('Date')
sales = sales[date_from:]

sales_exog = sales[['Open','Promo','StateHoliday','SchoolHoliday']]
sales_exog['Open'] = pd.to_numeric(sales_exog['Open'], errors='coerce').fillna(0)
sales_exog['Promo'] = pd.to_numeric(sales_exog['Promo'], errors='coerce').fillna(0)
sales_exog['StateHoliday'] = pd.to_numeric(sales_exog['StateHoliday'], errors='coerce').fillna(0)
sales_exog['SchoolHoliday'] = pd.to_numeric(sales_exog['SchoolHoliday'], errors='coerce').fillna(0)

sales_exog = sales_exog[date_from:]

sales_exog = sales_exog.resample(rsmpl_opt).mean()
#sales_exog = sales_exog
sales_exog = sales_exog.fillna({'Open':0, 'Promo':0, 'StateHoliday':0, 'SchoolHoliday':0})
#print(sales_exog)



sales = sales['Sales'].resample(rsmpl_opt).mean()    


fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sales, lags=len(sales)/2-1, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sales, lags=len(sales)/2-1, ax=ax2)# , lags=40










In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sales, lags=len(sales)/2-1, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sales, lags=len(sales)/2-1, ax=ax2)# , lags=40

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries, window = 12, cutoff = 0.01):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window).mean()
    rolstd = timeseries.rolling(window).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', maxlag = 20 )
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    pvalue = dftest[1]
    if pvalue < cutoff:
        print('p-value = %.4f. The series is likely stationary.' % pvalue)
    else:
        print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
    print(dfoutput)

In [ ]:
test_stationarity(sales)

In [ ]:
first_diff = sales - sales.shift(1)
first_diff = first_diff.dropna(inplace = False)
test_stationarity(first_diff, window = 12)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(first_diff, lags=len(first_diff)-2,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(first_diff, lags=len(first_diff)/2-1, ax=ax2)

## MAIN SARIMAX LOOP FOR ALL STORE LIST

In [ ]:
hasil = pd.DataFrame()

# Change the rsmpl_opt and also spqd_period for different smoothing of data 
date_from = '2013-06-02'
date_to = ''
rsmpl_opt = 'M'
spqd_period = 12
program_starts = time.time()
iters = 0

for store in store_list:
    
   
    sales = train[train['Store'] == store]
    sales = sales.set_index('Date')
    sales = sales[date_from:]
    
    sales_exog = sales[['Open','Promo','StateHoliday','SchoolHoliday']]
    sales_exog['Open'] = pd.to_numeric(sales_exog['Open'], errors='coerce').fillna(0)
    sales_exog['Promo'] = pd.to_numeric(sales_exog['Promo'], errors='coerce').fillna(0)
    sales_exog['StateHoliday'] = pd.to_numeric(sales_exog['StateHoliday'], errors='coerce').fillna(0)
    sales_exog['SchoolHoliday'] = pd.to_numeric(sales_exog['SchoolHoliday'], errors='coerce').fillna(0)
    
    sales_exog = sales_exog[date_from:]
    
    sales_exog = sales_exog.resample(rsmpl_opt).mean()
    #sales_exog = sales_exog
    sales_exog = sales_exog.fillna({'Open':0, 'Promo':0, 'StateHoliday':0, 'SchoolHoliday':0})
    #print(sales_exog)
    
    
    
    sales = sales['Sales'].resample(rsmpl_opt).mean()    
    #y = pd.DataFrame(sales['Sales'])
    
    
    
    #params
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, [1], q))
    seasonal_pdq = [(x[0], 1, x[2], spqd_period) for x in pdq]
    #seasonal_pdq = [(0, 1, 1, spqd_period)]
    # IMPORTANTTTTT
    
    #p = q = range(0, 6) 
    #pdq = list(itertools.product([0], [1], q)) 
    #sp = sq = range(1,8)#range(0,1) <- ARIMAX 
    #seasonal_pdq = list(itertools.product(sp, [0,1], sq,[1]))#rlist(itertools.product(sp, [0], sq,[0]))<- ARIMAX
    
    
    #p = d = q = range(0, 6)
    #pdq = list(itertools.product(p, [1], q))
    #seasonal_pdq = [(x[0], x[1], x[2], spqd_period) for x in pdq]
    #seasonal_pdq = [(0, 1, 1, spqd_period)]
    
    
    
    #search for the best params
    aic = 9999999999999999999
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod2 = sm.tsa.statespace.SARIMAX(sales,
                                                exog=sales_exog,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False,
                                                error_action='ignore',
                                                suppress_warnings=True)
                results = mod.fit()

                if ((results.aic < 50) && ) :
                    aic = results.aic
                    st = f'ARIMA{param}x{param_seasonal}12 - AIC:{results.aic}'
                    pl = param  #param with lowest aic
                    pls = param_seasonal #param_seasonal with lowest aic
                    mod = mod2
                    return
            except:
                continue
            

    #print('=========LOWEST AIC===========')
    
    #print(f'ARIMA{pl}x{pls}12 - AIC:{results.aic}')
    #print([pl[0], pl[1], pl[2]])
    #print([pls[0], pls[1], pls[2]])

    #mod = sm.tsa.statespace.SARIMAX(sales,
    #                                exog=sales_exog,
    #                                order=(pl[0], pl[1], pl[2]),
     #                               seasonal_order=(pls[0], pls[1], pls[2], spqd_period), # IMPORTANTTTTT
    #                                enforce_stationarity=False,
    #                                enforce_invertibility=False,
    #                              error_action='ignore',
     #                               suppress_warnings=True)
    results = mod.fit()
    #print(results.summary().tables[1])
    #print(results.summary().tables[1])

    
    #get the prediction for this store
    test_sales = test[test['Store'] == store]
    test_sales = test_sales.set_index('Date')
    test_sales_exog = test_sales[['Open','Promo','StateHoliday','SchoolHoliday']]
    
    test_sales_exog['Open'] = pd.to_numeric(test_sales_exog['Open'], errors='coerce').fillna(0)
    test_sales_exog['Promo'] = pd.to_numeric(test_sales_exog['Promo'], errors='coerce').fillna(0)
    test_sales_exog['StateHoliday'] = pd.to_numeric(test_sales_exog['StateHoliday'], errors='coerce').fillna(0)
    test_sales_exog['SchoolHoliday'] = pd.to_numeric(test_sales_exog['SchoolHoliday'], errors='coerce').fillna(0)

    test_sales_exog = test_sales_exog.resample(rsmpl_opt).mean()
    test_sales_exog = test_sales_exog.fillna({'Open':0, 'Promo':0, 'StateHoliday':0, 'SchoolHoliday':0})
    #print(test_sales_exog)
    
    
    pred_uc = results.get_forecast(steps=len(test_sales_exog),exog=test_sales_exog)
    pred_ci = pred_uc.conf_int()
    pred_ci = pd.DataFrame(pred_ci[['lower Sales','upper Sales']].mean(axis=1))
    pred_ci['Store'] = store
    pred_ci = pred_ci.sort_index()
    #pred_ci.sort_values(by=['Index'])
    hasil = hasil.append(pred_ci)
    
    
    
    
    
    #print(f'{sales_exog.index}')
    #print(f'{y.index} ')
    clear_output()
    print(st)
    iters = iters + 1
    print(f'done: {iters}/{len(store_list)}')
    now = time.time()
    print("It has been {0} seconds since the loop started".format(now - program_starts))

print('done')


    
    


In [ ]:
hasil2 = hasil

## Checking model

In [ ]:
date = str(sales_exog.last('10M').first('1d').index.item())
#pred = results.get_prediction(start=pd.to_datetime(date), exog=sales_exog[sales_exog.index >= pd.to_datetime(date)] , dynamic=False)
pred = results.get_prediction(start=pd.to_datetime(date), exog=sales_exog[sales_exog.index >= date] , dynamic=False)
pred_cix = pred.conf_int()
ax = sales[date_from:].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))

ax.fill_between(pred_cix.index,
                pred_cix.iloc[:, 0],
                pred_cix.iloc[:, 1], color='k', alpha=.2)
plt.legend()
plt.show()
pred.dist()

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = sales[date:]
mse = ((y_forecasted - y_truth) ** 2).mean()
print(f'The Mean Squared Error is {round(mse, 2)}')
print(f'The Root Mean Squared Error of our forecasts is {round(np.sqrt(mse))}')

In [ ]:
pd.concat([pd.DataFrame(sales), sales_exog],axis=1, join='inner')

In [ ]:
hasil

In [ ]:
store_list[1:25]

# Matching result and id, also turning the smoothed data to daily data

In [ ]:
wak = pd.DataFrame()
m = max(store_list)
for store in store_list:
    wak = wak.append(test[test['Store'] == store][['Id','Store','Date']])
    clear_output()
    print(f'{store}/{m}')
    
#wak
wkwk = pd.DataFrame([], columns=['Id','Date','Sales'])
iters = 0;
n = len(wak)
for index, row in wak.iterrows():
    wkwk = wkwk.append({'Id': row['Id'],'Date': row['Date'], 'Sales': hasil[hasil['Store'] == row['Store']].iloc[hasil[hasil['Store']== row['Store']].index.get_loc(row['Date'], method='nearest')][0]},ignore_index=True)
    iters = iters +1
    clear_output()
    print(f'matching = {iters}/{n}')
    


# Checking result and cleaning just before exporting the result into csv.

In [ ]:
wkwk2['Sales'][wkwk2['Sales'] > 10**3]

In [ ]:
wkwk2.describe()

In [ ]:
wkwk2 = wkwk[['Id', 'Sales']].sort_values(['Id'])
wkwk2
len(wkwk2['Sales'].unique())

In [ ]:
len(wkwk2['Sales'][np.isnan(wkwk2['Sales'])])
len(wkwk2['Sales'][wkwk2['Sales'] < 0])
wkwk2['Sales'][np.isnan(wkwk2['Sales'])] = wkwk2['Sales'].notnull().mean()
wkwk2['Sales'][wkwk2['Sales'] < 0] = wkwk2['Sales'].notnull().mean()
wkwk2['Sales'][wkwk2['Sales'] > 10**4] = wkwk2['Sales'].notnull().mean()


In [ ]:
wkwk2[np.isnan(wkwk2['Sales'])]

In [ ]:
wkwk2['Sales'][wkwk2['Sales'] < 0]

# OUTPUT to csv

In [ ]:
wkwk2.to_csv('weekly3.csv',index=False)